In [1]:
import pandas as pd
import tensorflow as tf
import numpy
import pickle as pkl
import os
os.chdir(os.path.join(os.getcwd(), ".."))
import finviz as fz
import yfinance as yf
import calendar
import datetime
import numpy as np
from data_layer import data_layer
from data_set import get_data_set_train
from data_set import get_data_set_test

In [2]:
perc_space = 0.8
perc_time = 0.8

WINDOW_SIZE = 21 # Include window + target
BATCH_SIZE = 8

data_path = os.path.join(os.getcwd(), "data")
sp_100_file = os.path.join(data_path, "sp-100-index-07-02-2020.csv")

# Load and Data formatting

In [3]:
stock = data_layer(sp_100_file, data_path, perc_space, perc_time)

BKNG <class 'IndexError'> list index out of range
BRK.B <class 'requests.exceptions.HTTPError'> 404 Client Error: Not Found for url: https://finviz.com/quote.ashx?t=BRK.B
C <class 'IndexError'> list index out of range
DD <class 'IndexError'> list index out of range
EXC <class 'IndexError'> list index out of range
GOOGL <class 'IndexError'> list index out of range
GS <class 'IndexError'> list index out of range
MO <class 'IndexError'> list index out of range
RTX <class 'IndexError'> list index out of range


In [4]:
n_train_space = int(len(stock.keys())*perc_space)
n_train_out_space = len(stock.keys())-n_train_space

In [5]:
train_space = np.random.choice(list(stock.keys()), size=n_train_space, replace=False).tolist()
test_space = list(set(stock.keys()) - set(train_space))
samples = stock["AAPL"].shape[0]
train_samples = int((samples-WINDOW_SIZE)*perc_time)

# Data loader object

In [16]:
train_ds, val_ds = get_data_set_train(WINDOW_SIZE,stock,train_space,train_samples)
test_ds = get_data_set_test(WINDOW_SIZE,stock,test_space)

Companies:  ['PM', 'AMGN', 'WFC', 'EMR', 'GD', 'QCOM', 'GILD', 'CRM', 'HD', 'KO', 'BK', 'AMT', 'ACN', 'KHC', 'DIS', 'PFE', 'TGT', 'BAC', 'LLY', 'GE', 'VZ', 'OXY', 'MCD', 'MSFT', 'BMY', 'ABBV', 'DUK', 'IBM', 'UPS', 'PEP', 'AAPL', 'BA', 'MET', 'CSCO', 'NKE', 'CHTR', 'F', 'SBUX', 'JPM', 'FDX', 'COF', 'MDT', 'UNH', 'LOW', 'TMO', 'ALL', 'BLK', 'T', 'MRK', 'MDLZ', 'HON', 'AMZN', 'INTC', 'JNJ', 'NEE', 'SO', 'DHR', 'UNP', 'FB', 'XOM', 'AXP', 'WMT', 'CVS', 'SLB', 'CAT', 'NFLX', 'TXN', 'MMM', 'BIIB', 'KMI', 'COST', 'ORCL', 'AIG']
Companies:  ['CL', 'NVDA', 'ADBE', 'DOW', 'GOOG', 'SPG', 'ABT', 'USB', 'MS', 'GM', 'CMCSA', 'MA', 'CVX', 'LMT', 'V', 'COP', 'PYPL', 'PG', 'WBA']


In [17]:
train_ds = train_ds.shuffle(buffer_size=5000)
train_ds = train_ds.batch(BATCH_SIZE)
val_ds = val_ds.batch(BATCH_SIZE)
test_ds = test_ds.batch(BATCH_SIZE)

# Model

In [9]:
def build_model():
    inputs = tf.keras.Input(shape=(20,10))
    x = tf.keras.layers.BatchNormalization(axis=2)(inputs)
    x = tf.keras.layers.GRU(64, return_sequences=False)(x)
    x = tf.keras.layers.Flatten()(x)
    sigmoids = tf.keras.layers.Dense(3, activation="sigmoid")(x)
    outputs = tf.keras.backend.expand_dims(sigmoids, axis=1)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [10]:
model = build_model()
model.compile(loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy", "AUC"])

In [20]:
history = model.fit(train_ds, 
          validation_data=val_ds,
          epochs=3)

Epoch 1/3
24/24 [==============================] - 2s 97ms/step - loss: 0.6260 - accuracy: 0.8324 - AUC: 0.3623 - val_loss: 0.8263 - val_accuracy: 0.6667 - val_AUC: 0.0000e+00
Epoch 2/3
24/24 [==============================] - 0s 13ms/step - loss: 0.2278 - accuracy: 0.9892 - AUC: 0.6547 - val_loss: 0.3875 - val_accuracy: 1.0000 - val_AUC: 0.0000e+00
Epoch 3/3
24/24 [==============================] - 0s 13ms/step - loss: 0.0387 - accuracy: 0.9928 - AUC: 0.7704 - val_loss: 0.1751 - val_accuracy: 1.0000 - val_AUC: 0.0000e+00


In [ ]:
model.evaluate(test_ds)

# Comparing predictions and looking if it is learning anything
dataframe start from the 20, example df[20:] are all the samples in test, and each company has 232 records, so to iterate would be accooording to 232 and addding 20 to the origiinal df

In [22]:
out = model.predict(test_ds)
predicted_df = pd.DataFrame(np.squeeze(out), columns=["Buy", "Sell", "Option Exercise"])
for company in test_space:
    print("Company: ", company)
    curr_hist = stock[company]
    break